In [1]:
import numpy as np
import cv2
import os

In [2]:
# Function to handle mouse events
def draw_line(event, x, y, flags, param):
    global drawing, ix, iy, start_points, end_points
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        start_points.append((ix, iy))
        end_points.append((x, y))

In [3]:
# Input video filename
input_filename = 'Ref_video_T026.mp4'

# Paths and directories
working_directory = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/nadia_distance_estimation/'
input_video_path = os.path.join(working_directory, 'videos', input_filename)
output_video_suffix = '_output.mp4'

# Output folder for modified videos and arrays
output_folder = os.path.join(working_directory, 'output')
os.makedirs(output_folder, exist_ok=True)

# Load saved start and end points arrays if they exist
start_points_path = os.path.join(output_folder, os.path.splitext(input_filename)[0] + '_start_points.npy')
end_points_path = os.path.join(output_folder, os.path.splitext(input_filename)[0] + '_end_points.npy')

if os.path.exists(start_points_path) and os.path.exists(end_points_path):
    start_points_array = np.load(start_points_path)
    end_points_array = np.load(end_points_path)
else:
    start_points_array = None
    end_points_array = None

In [4]:
# Codec for video encoding
fourcc = cv2.VideoWriter_fourcc(*'avc1')

# Open the input video file
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create the output video file
output_filename = os.path.splitext(input_filename)[0] + output_video_suffix
output_video_path = os.path.join(output_folder, output_filename)
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height), True)

# Create a window for displaying frames
cv2.namedWindow('Frame')

# Set the mouse callback function
cv2.setMouseCallback('Frame', draw_line)

drawing = False
ix, iy = -1, -1
start_points = start_points_array.tolist() if start_points_array is not None else []
end_points = end_points_array.tolist() if end_points_array is not None else []

# Process each frame in the input video
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    for start, end in zip(start_points, end_points):
        cv2.line(frame, tuple(start), tuple(end), (0, 255, 0), 2)
    
    if drawing:
        for start, end in zip(start_points, end_points):
            cv2.line(frame, tuple(start), tuple(end), (0, 255, 0), 2)
        cv2.line(frame, (ix, iy), (ix, iy), (0, 255, 0), 2)

    cv2.imshow('Frame', frame)
    out.write(frame)

    if cv2.waitKey(1) == 27:
        break

# Convert the lists of start and end points to NumPy arrays
start_points_array = np.array(start_points)
end_points_array = np.array(end_points)

# Save the arrays to files in the output folder
np.save(start_points_path, start_points_array)
np.save(end_points_path, end_points_array)

cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1